## Imbalanced class metrics
- Class imbalance is something that can hamper your model's performance in any machine learning context. This is especially relevant in a machine learning interview if we are asked what to do if we are given a dataset with an imbalanced class, as some data is imbalanced by design such as insurance fraud data.
- We'll use sklearn to create a logistic regression model and print the confusion matrix along with several evaluation metrics to get a better understanding of how to interpret Machine Learning models from datasets that have a class imbalance.

In [19]:
# Import
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [15]:
data = pd.read_csv('data/LoanTrainingSetScaledAndOHE.csv')
print(data.columns)
# print(data.head())

Index(['Loan Status', 'Current Loan Amount', 'Credit Score',
       'Years in current job', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens', 'DTI Ratio',
       'Term_Long Term', 'Term_Short Term', 'Home Ownership_Mortgage',
       'Home Ownership_Own Home', 'Home Ownership_Rent',
       'Purpose_BusinessLoan', 'Purpose_CarPurchase',
       'Purpose_DebtConsolidation', 'Purpose_EducationalExpenses',
       'Purpose_HomeImprovements', 'Purpose_HomePurchase',
       'Purpose_MajorPurchase', 'Purpose_MedicalBills', 'Purpose_Moving',
       'Purpose_Other', 'Purpose_RenewableEnergyPurchase', 'Purpose_Vacation',
       'Purpose_Wedding'],
      dtype='object')


In [16]:
data['Loan Status'].value_counts()

1    35809
0    14191
Name: Loan Status, dtype: int64

In [17]:
# import data and split
X = data.drop('Loan Status', axis=1)
y = data['Loan Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=123)


# Instantiate, fit, predict
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# Print evaluation metrics
print("Confusion matrix:\n {}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))
print("Precision: {}".format(precision_score(y_test, y_pred)))
print("Recall: {}".format(recall_score(y_test, y_pred)))
print("F1: {}".format(f1_score(y_test, y_pred)))

Confusion matrix:
 [[  359  3898]
 [  291 10452]]
Accuracy: 0.7207333333333333
Precision: 0.728362369337979
Recall: 0.9729125942474169
F1: 0.8330610130315227


#### The metrics aren't actually too bad. Precision of 0.72 means there might be a high number of false positives than you really want to see. Let's see if you can improve them in the next exercise with some resampling techniques!

## Resampling techniques
- We saw how class imbalance can impact the results of our confusion matrix. Here, we'll practice resampling techniques to explore the different results that alternative resampling styles can have on a dataset with class imbalance like that seen with loan_data. Using sklearn's resample() function, matching the number of rows in the majority class is called upsampling, while matching the number of rows in the minority class is called downsampling. 
- **Task** : create both an upsampled and downsampled version of the loan_data dataset, apply a logistic regression on both of them and then evaluate your performance. The training data and its labels that correspond to deny are subset to contain only the minority class and to approve that correspond to the majority.

In [23]:
# The training data and its labels that correspond to deny are subset to
# contain only the minority class and to approve that correspond to the majority.

deny = data[data['Loan Status'] == 0]
print(deny['Loan Status'].value_counts())

approve = data[data['Loan Status'] == 1]
print(approve['Loan Status'].value_counts())

0    14191
Name: Loan Status, dtype: int64
1    35809
Name: Loan Status, dtype: int64


In [24]:
# Upsample minority and combine with majority
loans_upsampled = resample(deny, replace=True, n_samples=len(approve), random_state=123)
upsampled = pd.concat([approve, loans_upsampled])

# Downsample majority and combine with minority
loans_downsampled = resample(approve, replace = False,  n_samples = len(deny), random_state = 123)
downsampled = pd.concat([loans_downsampled, deny])

- Create an upsampled feature matrix and target array.
- Instantiate a logistic regression model object, fit, and predict with X_test.
- Print the evaluation metrics.

In [25]:
# Upsampled feature matrix and target array
X_train_up = upsampled.drop('Loan Status', axis=1)
y_train_up = upsampled['Loan Status']

# Instantiate, fit, predict
loan_lr_up = LogisticRegression(solver='liblinear')
loan_lr_up.fit(X_train_up, y_train_up)
upsampled_y_pred = loan_lr_up.predict(X_test)

# Print evaluation metrics
print("Confusion matrix:\n {}".format(confusion_matrix(y_test, upsampled_y_pred)))
print("Accuracy: {}".format(accuracy_score(y_test, upsampled_y_pred)))
print("Precision: {}".format(precision_score(y_test, upsampled_y_pred)))
print("Recall: {}".format(recall_score(y_test, upsampled_y_pred)))
print("F1: {}".format(f1_score(y_test, upsampled_y_pred)))

Confusion matrix:
 [[2373 1884]
 [3616 7127]]
Accuracy: 0.6333333333333333
Precision: 0.7909222061924315
Recall: 0.6634087312668715
F1: 0.7215753771388073


- Create a downsampled feature matrix and target array.
- Instantiate a logistic regression model object, fit, and predict with X_test.
- Print the evaluation metrics.


In [26]:
# Downsampled feature matrix and target array
X_train_down = downsampled.drop('Loan Status', axis=1)
y_train_down = downsampled['Loan Status']

# Instantiate, fit, predict
loan_lr_down = LogisticRegression(solver='liblinear')
loan_lr_down.fit(X_train_down, y_train_down)
downsampled_y_pred = loan_lr_down.predict(X_test)

# Print evaluation metrics
print("Confusion matrix:\n {}".format(confusion_matrix(y_test, downsampled_y_pred)))
print("Accuracy: {}".format(accuracy_score(y_test, downsampled_y_pred)))
print("Precision: {}".format(precision_score(y_test, downsampled_y_pred)))
print("Recall: {}".format(recall_score(y_test, downsampled_y_pred)))
print("F1: {}".format(f1_score(y_test, downsampled_y_pred)))

Confusion matrix:
 [[2380 1877]
 [3695 7048]]
Accuracy: 0.6285333333333334
Precision: 0.7896918767507003
Recall: 0.6560551056501909
F1: 0.7166971730730122


#### Using both upsampling and downsampling techniques improved the precision score significantly, meaning there are less false positives. That is definitely a good thing!